In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [5]:
dataset = pd.read_csv(r'C:\Users\VIVEK\Downloads\Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [7]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [8]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [9]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [10]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [11]:
#Standardize the features
X = pipeline.fit_transform(X)

In [12]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [13]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [14]:
#Initialize ANN
classifier = Sequential()

In [15]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [16]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [17]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [18]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 1s - loss: 0.6890 - accuracy: 0.6072 - val_loss: 0.5368 - val_accuracy: 0.7675 - 1s/epoch - 6ms/step
Epoch 2/200
225/225 - 0s - loss: 0.5202 - accuracy: 0.7836 - val_loss: 0.4696 - val_accuracy: 0.8050 - 445ms/epoch - 2ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4750 - accuracy: 0.8018 - val_loss: 0.4413 - val_accuracy: 0.8075 - 336ms/epoch - 1ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4651 - accuracy: 0.8014 - val_loss: 0.4242 - val_accuracy: 0.8138 - 234ms/epoch - 1ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4538 - accuracy: 0.8056 - val_loss: 0.4114 - val_accuracy: 0.8200 - 235ms/epoch - 1ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4388 - accuracy: 0.8099 - val_loss: 0.4023 - val_accuracy: 0.8225 - 246ms/epoch - 1ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4374 - accuracy: 0.8104 - val_loss: 0.3977 - val_accuracy: 0.8200 - 235ms/epoch - 1ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4284 - accuracy: 0.8157 - val_loss: 0.3914 - val_accuracy: 0.8313 - 252ms/epoch - 1ms/

Epoch 66/200
225/225 - 0s - loss: 0.3710 - accuracy: 0.8474 - val_loss: 0.3394 - val_accuracy: 0.8600 - 242ms/epoch - 1ms/step
Epoch 67/200
225/225 - 0s - loss: 0.3717 - accuracy: 0.8467 - val_loss: 0.3420 - val_accuracy: 0.8612 - 255ms/epoch - 1ms/step
Epoch 68/200
225/225 - 0s - loss: 0.3687 - accuracy: 0.8481 - val_loss: 0.3412 - val_accuracy: 0.8625 - 259ms/epoch - 1ms/step
Epoch 69/200
225/225 - 0s - loss: 0.3724 - accuracy: 0.8490 - val_loss: 0.3412 - val_accuracy: 0.8625 - 255ms/epoch - 1ms/step
Epoch 70/200
225/225 - 0s - loss: 0.3739 - accuracy: 0.8471 - val_loss: 0.3388 - val_accuracy: 0.8600 - 266ms/epoch - 1ms/step
Epoch 71/200
225/225 - 0s - loss: 0.3709 - accuracy: 0.8475 - val_loss: 0.3407 - val_accuracy: 0.8575 - 226ms/epoch - 1ms/step
Epoch 72/200
225/225 - 0s - loss: 0.3691 - accuracy: 0.8478 - val_loss: 0.3404 - val_accuracy: 0.8600 - 238ms/epoch - 1ms/step
Epoch 73/200
225/225 - 0s - loss: 0.3674 - accuracy: 0.8499 - val_loss: 0.3391 - val_accuracy: 0.8600 - 232ms/e

Epoch 131/200
225/225 - 1s - loss: 0.3684 - accuracy: 0.8467 - val_loss: 0.3377 - val_accuracy: 0.8600 - 1s/epoch - 5ms/step
Epoch 132/200
225/225 - 1s - loss: 0.3659 - accuracy: 0.8481 - val_loss: 0.3365 - val_accuracy: 0.8625 - 802ms/epoch - 4ms/step
Epoch 133/200
225/225 - 1s - loss: 0.3698 - accuracy: 0.8465 - val_loss: 0.3406 - val_accuracy: 0.8575 - 796ms/epoch - 4ms/step
Epoch 134/200
225/225 - 1s - loss: 0.3695 - accuracy: 0.8449 - val_loss: 0.3404 - val_accuracy: 0.8575 - 974ms/epoch - 4ms/step
Epoch 135/200
225/225 - 1s - loss: 0.3649 - accuracy: 0.8493 - val_loss: 0.3353 - val_accuracy: 0.8625 - 845ms/epoch - 4ms/step
Epoch 136/200
225/225 - 1s - loss: 0.3651 - accuracy: 0.8474 - val_loss: 0.3388 - val_accuracy: 0.8600 - 1s/epoch - 5ms/step
Epoch 137/200
225/225 - 1s - loss: 0.3670 - accuracy: 0.8478 - val_loss: 0.3394 - val_accuracy: 0.8612 - 1s/epoch - 5ms/step
Epoch 138/200
225/225 - 1s - loss: 0.3632 - accuracy: 0.8483 - val_loss: 0.3365 - val_accuracy: 0.8612 - 742ms/ep

225/225 - 0s - loss: 0.3625 - accuracy: 0.8494 - val_loss: 0.3374 - val_accuracy: 0.8587 - 234ms/epoch - 1ms/step
Epoch 196/200
225/225 - 0s - loss: 0.3653 - accuracy: 0.8493 - val_loss: 0.3405 - val_accuracy: 0.8587 - 291ms/epoch - 1ms/step
Epoch 197/200
225/225 - 0s - loss: 0.3627 - accuracy: 0.8499 - val_loss: 0.3397 - val_accuracy: 0.8575 - 249ms/epoch - 1ms/step
Epoch 198/200
225/225 - 0s - loss: 0.3637 - accuracy: 0.8496 - val_loss: 0.3397 - val_accuracy: 0.8562 - 252ms/epoch - 1ms/step
Epoch 199/200
225/225 - 0s - loss: 0.3615 - accuracy: 0.8478 - val_loss: 0.3374 - val_accuracy: 0.8562 - 241ms/epoch - 1ms/step
Epoch 200/200
225/225 - 0s - loss: 0.3653 - accuracy: 0.8490 - val_loss: 0.3377 - val_accuracy: 0.8575 - 273ms/epoch - 1ms/step


In [21]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 967us/step
[[0.27264768]
 [0.20741305]
 [0.18282363]
 [0.08801566]
 [0.08770575]]


In [22]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [23]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1558   37]
 [ 242  163]]


In [24]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.05 % of data was classified correctly
